<a href="https://colab.research.google.com/github/skyprince999/AV-GOT-Ship-Identification/blob/master/AV_GoT_Ship_Baseline_Model_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL.Image as Image
from IPython.display import clear_output

TensorFlow version is  1.13.1


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/my_modules')
import unzipFiles

In [0]:
PATH = '/content/gdrive/My Drive/AV_GoT_Ships/'

categories = {'Cargo': 1, 
'Military': 2, 
'Carrier': 3, 
'Cruise': 4, 
'Tankers': 5}

os.listdir(PATH)

['train.zip',
 'test.csv',
 'sample_submission.csv',
 'models',
 'submission_mobilenet_25May2019_2335.csv',
 'training_1',
 'training_2',
 'training_3',
 'models_v2']

In [0]:
unzipFiles.copyTest(PATH)
unzipFiles.unzipData(PATH)
unzipFiles.copyTrainValid(categories)

['.config', 'sample_submission.csv', 'test.csv', 'gdrive', 'sample_data']
Unzipped Successful
['.config', 'sample_submission.csv', 'images', 'test.csv', '__MACOSX', 'train.csv', 'gdrive', 'sample_data']
['.config', 'sample_submission.csv', 'images', 'test.csv', '__MACOSX', 'train.csv', 'gdrive', 'sample_data']
Shape of train file:  (6252, 2)
Number of categories:  5


0

In [0]:
image_size = 160 # All images will be resized to 160x160
batch_size = 32
train_dir = 'train'
validation_dir = 'valid'


# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [0]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size, seed = 999,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size, seed = 999,
                class_mode='categorical')

Found 4999 images belonging to 5 classes.
Found 1253 images belonging to 5 classes.


In [0]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 0s 0us/step


In [0]:
base_model.trainable = False

In [0]:
# Let's take a look at the base model architecture
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(5, activation='sigmoid')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
len(model.trainable_variables)

2

In [0]:
epochs = 5
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

In [0]:
checkpoint_path = PATH + "training_1/cp_v2.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, period = 2,
                                                 verbose=1)

In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                             callbacks = [cp_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
157/157 [==============================] - 17s 106ms/step - loss: 1.4936 - categorical_accuracy: 0.3343 - val_loss: 1.5297 - val_categorical_accuracy: 0.3392
Epoch 2/5
40/40 [==============================] - 2s 60ms/step - loss: 1.3159 - categorical_accuracy: 0.4397

Epoch 00002: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_1/cp_v2.ckpt
157/157 [==============================] - 12s 79ms/step - loss: 1.2025 - categorical_accuracy: 0.5223 - val_loss: 1.3159 - val_categorical_accuracy: 0.4397
Epoch 3/5
157/157 [==============================] - 12s 79ms/step - loss: 0.9239 - categorical_accuracy: 0.6775 - val_loss: 1.1574 - val_categorical_accuracy: 0.5387
Epoch 4/5
40/40 [==============================] - 2s 60ms/step - loss: 0.9939 - categorical_accuracy: 0.6297

Epoch 00004: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_1/cp_v2.ckpt
157/157 [==============================] - 12s 77ms/step - l

In [0]:
os.listdir(PATH + 'training_1')

['cp.ckpt', 'cp_v2.ckpt']

In [0]:
model.save(PATH + 'models_v2/epoch5_26May2019_1437.h5')

In [0]:
epochs = 15
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

checkpoint_path = PATH + "training_2/cp_v2.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 period = 2,
                                                 verbose=1)

In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                             callbacks = [cp_callback])

Epoch 1/15
157/157 [==============================] - 12s 75ms/step - loss: 0.6495 - categorical_accuracy: 0.7720 - val_loss: 0.8108 - val_categorical_accuracy: 0.6903
Epoch 2/15
40/40 [==============================] - 2s 60ms/step - loss: 0.7270 - categorical_accuracy: 0.7334

Epoch 00002: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_2/cp_v2.ckpt
157/157 [==============================] - 12s 73ms/step - loss: 0.6151 - categorical_accuracy: 0.7776 - val_loss: 0.7270 - val_categorical_accuracy: 0.7334
Epoch 3/15
157/157 [==============================] - 11s 73ms/step - loss: 0.5929 - categorical_accuracy: 0.7878 - val_loss: 0.6591 - val_categorical_accuracy: 0.7574
Epoch 4/15
40/40 [==============================] - 2s 58ms/step - loss: 0.6307 - categorical_accuracy: 0.7749

Epoch 00004: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_2/cp_v2.ckpt
157/157 [==============================] - 12s 77ms/step - loss: 0.5668 - categorical_accuracy: 0.7936 - 

In [0]:
model.save(PATH + 'models_v2/epoch15_26May2019_1445.h5')

In [0]:
epochs = 25
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

checkpoint_path = PATH + "training_3/cp_v2.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 period = 2,
                                                 verbose=1)

In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                             callbacks = [cp_callback])

Epoch 1/25
157/157 [==============================] - 12s 73ms/step - loss: 0.4462 - categorical_accuracy: 0.8356 - val_loss: 0.5144 - val_categorical_accuracy: 0.8005
Epoch 2/25
40/40 [==============================] - 2s 60ms/step - loss: 0.5101 - categorical_accuracy: 0.7973

Epoch 00002: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_3/cp_v2.ckpt
157/157 [==============================] - 11s 73ms/step - loss: 0.4399 - categorical_accuracy: 0.8390 - val_loss: 0.5101 - val_categorical_accuracy: 0.7973
Epoch 3/25
157/157 [==============================] - 12s 75ms/step - loss: 0.4327 - categorical_accuracy: 0.8420 - val_loss: 0.5091 - val_categorical_accuracy: 0.8021
Epoch 4/25
40/40 [==============================] - 2s 61ms/step - loss: 0.5150 - categorical_accuracy: 0.7973

Epoch 00004: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_3/cp_v2.ckpt
157/157 [==============================] - 13s 80ms/step - loss: 0.4311 - categorical_accuracy: 0.8446 - 

In [0]:
model.save(PATH + 'models_v2/epoch25_26May2019_1500.h5')

In [0]:
import pandas as pd

allDF = list()
pred = dict()
pred['Cargo'] = 0.8
pred['Carrier'] = 0.1
pred['Cruise'] = 0.3
pred['Military'] = 0.1
pred['Tankers'] = 0.24
allDF.append(pred)


pred = dict()
pred['Cargo'] = 0.4
pred['Carrier'] = 0.3
pred['Cruise'] = 0.4
pred['Military'] = 0.7
pred['Tankers'] = 0.15
allDF.append(pred)

pred = dict()
pred['Cargo'] = 0.2
pred['Carrier'] = 0.04
pred['Cruise'] = 0.1
pred['Military'] = 0.4
pred['Tankers'] = 0.6
allDF.append(pred)

pred = dict()
pred['Cargo'] = 0.12
pred['Carrier'] = 0.15
pred['Cruise'] = 0.26
pred['Military'] = 0.11
pred['Tankers'] = 0.56
allDF.append(pred)

all_data =pd.DataFrame(allDF)

In [0]:
import numpy as np

In [0]:
all_data.drop(labels = ['category'], inplace=True, axis = 1)
all_data

,Cargo,Carrier,Cruise,Military,Tankers
0,0.80,0.10,0.30,0.10,0.24
1,0.40,0.30,0.40,0.70,0.15
2,0.20,0.04,0.10,0.40,0.60
3,0.12,0.15,0.26,0.11,0.56


In [0]:
all_data['category'] = all_data.apply( np.argmax, axis = 1)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


In [0]:
all_data

,Cargo,Carrier,Cruise,Military,Tankers,category
0,0.80,0.10,0.30,0.10,0.24,Cargo
1,0.40,0.30,0.40,0.70,0.15,Military
2,0.20,0.04,0.10,0.40,0.60,Tankers
3,0.12,0.15,0.26,0.11,0.56,Tankers


In [0]:
all_data[['Cargo', 'Carrier', 'Cruise', 'Military', 'Tankers']].apply(np.argmax, axis=1)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


0       Cargo
1    Military
2     Tankers
3     Tankers
dtype: object